In [30]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_selection import f_classif , chi2

from sklearn.linear_model import LogisticRegression

from sklearn.preprocessing import StandardScaler 
from sklearn import metrics

from sklearn.datasets import load_breast_cancer
from sklearn.feature_selection import RFE 
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
import openml


In [31]:
data = load_breast_cancer()
X = pd.DataFrame(data.data , columns=data.feature_names)
y = pd.Series(data.target)


In [32]:
X_train , X_test , y_train , y_test = train_test_split(X,y,test_size=0.2 , random_state=42)


In [34]:
rf_model = RandomForestClassifier(random_state=42)
rfe = RFE(estimator=rf_model, n_features_to_select=13)
rfe.fit(X_train, y_train)

,"estimator estimator: ``Estimator`` instanceA supervised learning estimator with a ``fit`` method that providesinformation about feature importance(e.g. `coef_`, `feature_importances_`).",RandomForestC...ndom_state=42)
,"n_features_to_select n_features_to_select: int or float, default=NoneThe number of features to select. If `None`, half of the features areselected. If integer, the parameter is the absolute number of featuresto select. If float between 0 and 1, it is the fraction of features toselect... versionchanged:: 0.24 Added float values for fractions.",13
,"step step: int or float, default=1If greater than or equal to 1, then ``step`` corresponds to the(integer) number of features to remove at each iteration.If within (0.0, 1.0), then ``step`` corresponds to the percentage(rounded down) of features to remove at each iteration.",1
,"verbose verbose: int, default=0Controls verbosity of output.",0
,"importance_getter importance_getter: str or callable, default='auto'If 'auto', uses the feature importance either through a `coef_`or `feature_importances_` attributes of estimator.Also accepts a string that specifies an attribute name/pathfor extracting feature importance (implemented with `attrgetter`).For example, give `regressor_.coef_` in case of:class:`~sklearn.compose.TransformedTargetRegressor` or`named_steps.clf.feature_importances_` in case ofclass:`~sklearn.pipeline.Pipeline` with its last step named `clf`.If `callable`, overrides the default feature importance getter.The callable is passed with the fitted estimator and it shouldreturn importance for each feature... versionadded:: 0.24",'auto'
,"n_estimators n_estimators: int, default=100The number of trees in the forest... versionchanged:: 0.22 The default value of ``n_estimators`` changed from 10 to 100 in 0.22.",100
,"criterion criterion: {""gini"", ""entropy"", ""log_loss""}, default=""gini""The function to measure the quality of a split. Supported criteria are""gini"" for the Gini impurity and ""log_loss"" and ""entropy"" both for theShannon information gain, see :ref:`tree_mathematical_formulation`.Note: This parameter is tree-specific.",'gini'
,"max_depth max_depth: int, default=NoneThe maximum depth of the tree. If None, then nodes are expanded untilall leaves are pure or until all leaves contain less thanmin_samples_split samples.",None
,"min_samples_split min_samples_split: int or float, default=2The minimum number of samples required to split an internal node:- If int, then consider `min_samples_split` as the minimum number.- If float, then `min_samples_split` is a fraction and `ceil(min_samples_split * n_samples)` are the minimum number of samples for each split... versionchanged:: 0.18 Added float values for fractions.",2
,"min_samples_leaf min_samples_leaf: int or float, default=1The minimum number of samples required to be at a leaf node.A split point at any depth will only be considered if it leaves atleast ``min_samples_leaf`` training samples in each of the left andright branches. This may have the effect of smoothing the model,especially in regression.- If int, then consider `min_samples_leaf` as the minimum number.- If float, then `min_samples_leaf` is a fraction and `ceil(min_samples_leaf * n_samples)` are the minimum number of samples for each node... versionchanged:: 0.18 Added float values for fractions.",1
,"min_weight_fraction_leaf min_weight_fraction_leaf: float, default=0.0The minimum weighted fraction of the sum total of weights (of allthe input samples) required to be at a leaf node. Samples haveequal weight when sample_weight is not provided.",0.0


In [35]:
selected_features = X_train.columns[rfe.support_]
selected_features

Index(['mean radius', 'mean texture', 'mean perimeter', 'mean area',
       'mean concavity', 'mean concave points', 'worst radius',
       'worst texture', 'worst perimeter', 'worst area', 'worst smoothness',
       'worst concavity', 'worst concave points'],
      dtype='object')

In [36]:
X_train_rfe_rf = rfe.transform(X_train)
X_test_rfe_rf = rfe.transform(X_test)

In [37]:
rf_model.fit(X_train_rfe_rf , y_train)

,"n_estimators n_estimators: int, default=100The number of trees in the forest... versionchanged:: 0.22 The default value of ``n_estimators`` changed from 10 to 100 in 0.22.",100
,"criterion criterion: {""gini"", ""entropy"", ""log_loss""}, default=""gini""The function to measure the quality of a split. Supported criteria are""gini"" for the Gini impurity and ""log_loss"" and ""entropy"" both for theShannon information gain, see :ref:`tree_mathematical_formulation`.Note: This parameter is tree-specific.",'gini'
,"max_depth max_depth: int, default=NoneThe maximum depth of the tree. If None, then nodes are expanded untilall leaves are pure or until all leaves contain less thanmin_samples_split samples.",None
,"min_samples_split min_samples_split: int or float, default=2The minimum number of samples required to split an internal node:- If int, then consider `min_samples_split` as the minimum number.- If float, then `min_samples_split` is a fraction and `ceil(min_samples_split * n_samples)` are the minimum number of samples for each split... versionchanged:: 0.18 Added float values for fractions.",2
,"min_samples_leaf min_samples_leaf: int or float, default=1The minimum number of samples required to be at a leaf node.A split point at any depth will only be considered if it leaves atleast ``min_samples_leaf`` training samples in each of the left andright branches. This may have the effect of smoothing the model,especially in regression.- If int, then consider `min_samples_leaf` as the minimum number.- If float, then `min_samples_leaf` is a fraction and `ceil(min_samples_leaf * n_samples)` are the minimum number of samples for each node... versionchanged:: 0.18 Added float values for fractions.",1
,"min_weight_fraction_leaf min_weight_fraction_leaf: float, default=0.0The minimum weighted fraction of the sum total of weights (of allthe input samples) required to be at a leaf node. Samples haveequal weight when sample_weight is not provided.",0.0
,"max_features max_features: {""sqrt"", ""log2"", None}, int or float, default=""sqrt""The number of features to consider when looking for the best split:- If int, then consider `max_features` features at each split.- If float, then `max_features` is a fraction and `max(1, int(max_features * n_features_in_))` features are considered at each split.- If ""sqrt"", then `max_features=sqrt(n_features)`.- If ""log2"", then `max_features=log2(n_features)`.- If None, then `max_features=n_features`... versionchanged:: 1.1 The default of `max_features` changed from `""auto""` to `""sqrt""`.Note: the search for a split does not stop until at least onevalid partition of the node samples is found, even if it requires toeffectively inspect more than ``max_features`` features.",'sqrt'
,"max_leaf_nodes max_leaf_nodes: int, default=NoneGrow trees with ``max_leaf_nodes`` in best-first fashion.Best nodes are defined as relative reduction in impurity.If None then unlimited number of leaf nodes.",None
,"min_impurity_decrease min_impurity_decrease: float, default=0.0A node will be split if this split induces a decrease of the impuritygreater than or equal to this value.The weighted impurity decrease equation is the following:: N_t / N * (impurity - N_t_R / N_t * right_impurity - N_t_L / N_t * left_impurity)where ``N`` is the total number of samples, ``N_t`` is the number ofsamples at the current node, ``N_t_L`` is the number of samples in theleft child, and ``N_t_R`` is the number of samples in the right child.``N``, ``N_t``, ``N_t_R`` and ``N_t_L`` all refer to the weighted sum,if ``sample_weight`` is passed... versionadded:: 0.19",0.0
,"bootstrap bootstrap: bool, default=TrueWhether bootstrap samples are used when building trees. If False, thewhole dataset is used to build each tree.",True
,"oob_score oob_score: bool or callable, default=FalseWhether to use out-of-bag samples to estimate the generalization score.By default, :func:`~sklearn.metrics.accuracy_score` is used.Provide a callable with signature `metric

In [38]:
print(metrics.classification_report(y_test,rf_model.predict(X_test_rfe_rf)))

              precision    recall  f1-score   support

           0       0.98      0.93      0.95        43
           1       0.96      0.99      0.97        71

    accuracy                           0.96       114
   macro avg       0.97      0.96      0.96       114
weighted avg       0.97      0.96      0.96       114



In [39]:
dataset = openml.datasets.get_dataset(1477)

In [40]:
X,y,_,_ = dataset.get_data(dataset_format='dataframe',target=dataset.default_target_attribute)

In [41]:
X.head()

,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,...,V120,V121,V122,V123,V124,V125,V126,V127,V128,V129
0,12285.6582,4.076635,4.842317,7.509393,10.822436,-1.312657,-1.853717,-6.924985,11800.9233,4.483500,...,-3.440387,1784.5324,1.907000,1.729200,4.881194,8.623828,-0.314110,-0.661556,-3.521663,10.0
1,-35.6889,0.993944,0.166099,0.489363,3.484663,-0.130298,-0.528364,-3.735347,266.4145,1.053988,...,-4.482534,904.9898,1.433707,1.068069,2.532958,5.369720,-0.183779,-0.534087,-4.635975,50.0
2,63927.2217,14.956941,19.971376,29.188512,33.291320,-10.433776,-16.062245,-49.490143,57405.8483,15.613843,...,-25.150090,14585.7879,8.189021,6.099452,12.127991,15.709651,-3.887082,-6.731473,-19.326895,250.0
3,2992.9019,1.380553,0.808910,1.288259,4.660135,-0.755903,-1.120470,-4.075213,4301.4033,1.652701,...,-7.215792,6044.5554,3.488295,2.662288,5.938297,8.544508,-1.567322,-2.701235,-6.472439,600.0
4,57524.7812,11.912566,14.631496,19.809240,23.715868,-9.084750,-11.770585,-39.234003,50051.0703,11.732548,...,-10.248794,10580.1006,5.752675,3.880740,8.545897,11.831716,-2.655521,-4.312744,-8.510591,150.0


In [42]:
def drop_redundant_features(X,threshold=0.9 , method='pearson'):
    corr_matrix = X.corr(method=method)
    redundant_pairs = set()
    for i in range(len(corr_matrix.columns)):
        for j in range(i):    # loop only through lower triangle
            if abs(corr_matrix.iloc[i, j]) > threshold:
                colname1 = corr_matrix.columns[i]
                colname2 = corr_matrix.columns[j]
                redundant_pairs.add((colname1, colname2))
    print(f"Redundant feature pairs (threshold={threshold}): {redundant_pairs}")
    features_to_drop = set()
    for pair in redundant_pairs:
        features_to_drop.add(pair[0])
    X_reduced = X.drop(columns=features_to_drop)
    return X_reduced, features_to_drop

X_reduced, dropped_features = drop_redundant_features(X, threshold=0.9, method='pearson')
print(f"Dropped features using function: {dropped_features}")

Redundant feature pairs (threshold=0.9): {('V12', 'V3'), ('V96', 'V88'), ('V121', 'V81'), ('V95', 'V30'), ('V14', 'V11'), ('V85', 'V59'), ('V92', 'V91'), ('V85', 'V52'), ('V117', 'V91'), ('V77', 'V75'), ('V61', 'V19'), ('V67', 'V3'), ('V89', 'V22'), ('V126', 'V55'), ('V81', 'V51'), ('V81', 'V49'), ('V73', 'V1'), ('V52', 'V51'), ('V71', 'V70'), ('V89', 'V81'), ('V115', 'V30'), ('V60', 'V51'), ('V116', 'V85'), ('V84', 'V51'), ('V75', 'V4'), ('V7', 'V6'), ('V38', 'V33'), ('V94', 'V89'), ('V110', 'V102'), ('V86', 'V62'), ('V81', 'V25'), ('V51', 'V27'), ('V122', 'V18'), ('V87', 'V23'), ('V125', 'V60'), ('V119', 'V54'), ('V76', 'V9'), ('V94', 'V30'), ('V55', 'V54'), ('V102', 'V38'), ('V100', 'V99'), ('V51', 'V19'), ('V83', 'V27'), ('V89', 'V59'), ('V118', 'V30'), ('V86', 'V17'), ('V124', 'V53'), ('V41', 'V33'), ('V126', 'V86'), ('V50', 'V42'), ('V75', 'V12'), ('V14', 'V6'), ('V122', 'V90'), ('V91', 'V89'), ('V117', 'V93'), ('V68', 'V12'), ('V59', 'V27'), ('V119', 'V23'), ('V125', 'V61'), ('V

In [43]:
X_reduced.head()

,V1,V2,V8,V10,V13,V17,V18,V20,V21,V24,...,V40,V42,V47,V48,V56,V66,V98,V101,V104,V129
0,12285.6582,4.076635,-6.924985,4.483500,12.391131,1376.0439,1.617861,2.691709,5.254568,-2.692017,...,-2.419372,1.682275,-0.331580,-2.090179,-2.911391,3.747051,1.646146,5.722172,-3.800066,10.0
1,-35.6889,0.993944,-3.735347,1.053988,3.423493,1175.7173,1.522478,1.723403,4.208079,-3.299125,...,-2.891391,1.229708,-0.286360,-2.189327,-3.134272,0.998603,1.190208,4.290327,-3.366526,50.0
2,63927.2217,14.956941,-49.490143,15.613843,37.525114,12304.2205,6.392772,7.366502,9.704410,-10.335102,...,-4.384381,4.715781,-1.242500,-3.934958,-17.767853,11.568555,4.716139,7.836516,-5.561091,250.0
3,2992.9019,1.380553,-4.075213,1.652701,4.606337,7940.1109,4.105281,4.827317,7.013206,-5.350397,...,-2.431720,2.244966,-0.567184,-2.390613,-4.944249,1.314408,2.142778,4.697825,-4.510763,600.0
4,57524.7812,11.912566,-39.234003,11.732548,27.530928,8955.9827,4.671975,5.170638,7.308628,-5.966278,...,-3.006012,3.775566,-0.975634,-3.159041,-8.131812,8.785728,3.722984,7.119800,-4.455648,150.0


In [44]:
def anova_score_drop(X, y, threshold):
    anova_scores, _ = f_classif(X, y)
    selected_scores = anova_scores >= threshold
    X_anova = X.loc[:, selected_scores].copy()
    return X_anova
X_reduced_anova = anova_score_drop(X_reduced, y, threshold=10)
X_reduced_anova.head()

,V1,V2,V8,V10,V13,V17,V18,V20,V21,V24,...,V39,V40,V47,V48,V56,V66,V98,V101,V104,V129
0,12285.6582,4.076635,-6.924985,4.483500,12.391131,1376.0439,1.617861,2.691709,5.254568,-2.692017,...,-0.390274,-2.419372,-0.331580,-2.090179,-2.911391,3.747051,1.646146,5.722172,-3.800066,10.0
1,-35.6889,0.993944,-3.735347,1.053988,3.423493,1175.7173,1.522478,1.723403,4.208079,-3.299125,...,-0.350667,-2.891391,-0.286360,-2.189327,-3.134272,0.998603,1.190208,4.290327,-3.366526,50.0
2,63927.2217,14.956941,-49.490143,15.613843,37.525114,12304.2205,6.392772,7.366502,9.704410,-10.335102,...,-1.599611,-4.384381,-1.242500,-3.934958,-17.767853,11.568555,4.716139,7.836516,-5.561091,250.0
3,2992.9019,1.380553,-4.075213,1.652701,4.606337,7940.1109,4.105281,4.827317,7.013206,-5.350397,...,-0.715397,-2.431720,-0.567184,-2.390613,-4.944249,1.314408,2.142778,4.697825,-4.510763,600.0
4,57524.7812,11.912566,-39.234003,11.732548,27.530928,8955.9827,4.671975,5.170638,7.308628,-5.966278,...,-1.211382,-3.006012,-0.975634,-3.159041,-8.131812,8.785728,3.722984,7.119800,-4.455648,150.0


In [45]:
rf_model = RandomForestClassifier(random_state=42)
rfe = RFE(estimator=rf_model, n_features_to_select=10)
X_train , X_test , y_train , y_test = train_test_split(X_reduced_anova,y,test_size=0.2 , random_state=42)
#scaler = StandardScaler()
#X_train = scaler.fit_transform(X_train)
#X_test = scaler.transform(X_test)
rfe.fit(X_train, y_train)
selected_features = X_train.columns[rfe.support_]
selected_features

Index(['V1', 'V13', 'V17', 'V18', 'V20', 'V28', 'V36', 'V66', 'V101', 'V129'], dtype='object')

In [46]:
X_train_rfe_rf = rfe.transform(X_train)
X_test_rfe_rf = rfe.transform(X_test)

In [47]:
rf_model.fit(X_train_rfe_rf , y_train)

,"n_estimators n_estimators: int, default=100The number of trees in the forest... versionchanged:: 0.22 The default value of ``n_estimators`` changed from 10 to 100 in 0.22.",100
,"criterion criterion: {""gini"", ""entropy"", ""log_loss""}, default=""gini""The function to measure the quality of a split. Supported criteria are""gini"" for the Gini impurity and ""log_loss"" and ""entropy"" both for theShannon information gain, see :ref:`tree_mathematical_formulation`.Note: This parameter is tree-specific.",'gini'
,"max_depth max_depth: int, default=NoneThe maximum depth of the tree. If None, then nodes are expanded untilall leaves are pure or until all leaves contain less thanmin_samples_split samples.",None
,"min_samples_split min_samples_split: int or float, default=2The minimum number of samples required to split an internal node:- If int, then consider `min_samples_split` as the minimum number.- If float, then `min_samples_split` is a fraction and `ceil(min_samples_split * n_samples)` are the minimum number of samples for each split... versionchanged:: 0.18 Added float values for fractions.",2
,"min_samples_leaf min_samples_leaf: int or float, default=1The minimum number of samples required to be at a leaf node.A split point at any depth will only be considered if it leaves atleast ``min_samples_leaf`` training samples in each of the left andright branches. This may have the effect of smoothing the model,especially in regression.- If int, then consider `min_samples_leaf` as the minimum number.- If float, then `min_samples_leaf` is a fraction and `ceil(min_samples_leaf * n_samples)` are the minimum number of samples for each node... versionchanged:: 0.18 Added float values for fractions.",1
,"min_weight_fraction_leaf min_weight_fraction_leaf: float, default=0.0The minimum weighted fraction of the sum total of weights (of allthe input samples) required to be at a leaf node. Samples haveequal weight when sample_weight is not provided.",0.0
,"max_features max_features: {""sqrt"", ""log2"", None}, int or float, default=""sqrt""The number of features to consider when looking for the best split:- If int, then consider `max_features` features at each split.- If float, then `max_features` is a fraction and `max(1, int(max_features * n_features_in_))` features are considered at each split.- If ""sqrt"", then `max_features=sqrt(n_features)`.- If ""log2"", then `max_features=log2(n_features)`.- If None, then `max_features=n_features`... versionchanged:: 1.1 The default of `max_features` changed from `""auto""` to `""sqrt""`.Note: the search for a split does not stop until at least onevalid partition of the node samples is found, even if it requires toeffectively inspect more than ``max_features`` features.",'sqrt'
,"max_leaf_nodes max_leaf_nodes: int, default=NoneGrow trees with ``max_leaf_nodes`` in best-first fashion.Best nodes are defined as relative reduction in impurity.If None then unlimited number of leaf nodes.",None
,"min_impurity_decrease min_impurity_decrease: float, default=0.0A node will be split if this split induces a decrease of the impuritygreater than or equal to this value.The weighted impurity decrease equation is the following:: N_t / N * (impurity - N_t_R / N_t * right_impurity - N_t_L / N_t * left_impurity)where ``N`` is the total number of samples, ``N_t`` is the number ofsamples at the current node, ``N_t_L`` is the number of samples in theleft child, and ``N_t_R`` is the number of samples in the right child.``N``, ``N_t``, ``N_t_R`` and ``N_t_L`` all refer to the weighted sum,if ``sample_weight`` is passed... versionadded:: 0.19",0.0
,"bootstrap bootstrap: bool, default=TrueWhether bootstrap samples are used when building trees. If False, thewhole dataset is used to build each tree.",True
,"oob_score oob_score: bool or callable, default=FalseWhether to use out-of-bag samples to estimate the generalization score.By default, :func:`~sklearn.metrics.accuracy_score` is used.Provide a callable with signature `metric

In [48]:
print(metrics.classification_report(y_test,rf_model.predict(X_test_rfe_rf)))
print(metrics.accuracy_score(y_test,rf_model.predict(X_test_rfe_rf)))

              precision    recall  f1-score   support

           1       1.00      1.00      1.00       511
           2       1.00      1.00      1.00       587
           3       0.99      0.97      0.98       331
           4       0.99      0.99      0.99       379
           5       0.99      1.00      1.00       604
           6       0.99      1.00      0.99       370

    accuracy                           0.99      2782
   macro avg       0.99      0.99      0.99      2782
weighted avg       0.99      0.99      0.99      2782

0.9938892882818117


In [53]:

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

lr_model = LogisticRegression(max_iter=1000)
rfe_lr = RFE(estimator=lr_model, n_features_to_select=10)
rfe_lr.fit(X_train_scaled, y_train)
selected_features_lr = X_train.columns[rfe_lr.support_]
selected_features_lr

Index(['V2', 'V10', 'V13', 'V17', 'V18', 'V28', 'V39', 'V66', 'V98', 'V129'], dtype='object')

In [54]:
X_train_rfe_lr = rfe_lr.transform(X_train_scaled)
X_test_rfe_lr = rfe_lr.transform(X_test_scaled)

In [55]:
lr_model.fit(X_train_rfe_lr , y_train)

,"penalty penalty: {'l1', 'l2', 'elasticnet', None}, default='l2'Specify the norm of the penalty:- `None`: no penalty is added;- `'l2'`: add a L2 penalty term and it is the default choice;- `'l1'`: add a L1 penalty term;- `'elasticnet'`: both L1 and L2 penalty terms are added... warning:: Some penalties may not work with some solvers. See the parameter `solver` below, to know the compatibility between the penalty and solver... versionadded:: 0.19 l1 penalty with SAGA solver (allowing 'multinomial' + L1).. deprecated:: 1.8 `penalty` was deprecated in version 1.8 and will be removed in 1.10. Use `l1_ratio` instead. `l1_ratio=0` for `penalty='l2'`, `l1_ratio=1` for `penalty='l1'` and `l1_ratio` set to any float between 0 and 1 for `'penalty='elasticnet'`.",'deprecated'
,"C C: float, default=1.0Inverse of regularization strength; must be a positive float.Like in support vector machines, smaller values specify strongerregularization. `C=np.inf` results in unpenalized logistic regression.For a visual example on the effect of tuning the `C` parameterwith an L1 penalty, see::ref:`sphx_glr_auto_examples_linear_model_plot_logistic_path.py`.",1.0
,"l1_ratio l1_ratio: float, default=0.0The Elastic-Net mixing parameter, with `0 <= l1_ratio <= 1`. Setting`l1_ratio=1` gives a pure L1-penalty, setting `l1_ratio=0` a pure L2-penalty.Any value between 0 and 1 gives an Elastic-Net penalty of the form`l1_ratio * L1 + (1 - l1_ratio) * L2`... warning:: Certain values of `l1_ratio`, i.e. some penalties, may not work with some solvers. See the parameter `solver` below, to know the compatibility between the penalty and solver... versionchanged:: 1.8 Default value changed from None to 0.0... deprecated:: 1.8 `None` is deprecated and will be removed in version 1.10. Always use `l1_ratio` to specify the penalty type.",0.0
,"dual dual: bool, default=FalseDual (constrained) or primal (regularized, see also:ref:`this equation `) formulation. Dual formulationis only implemented for l2 penalty with liblinear solver. Prefer `dual=False`when n_samples > n_features.",False
,"tol tol: float, default=1e-4Tolerance for stopping criteria.",0.0001
,"fit_intercept fit_intercept: bool, default=TrueSpecifies if a constant (a.k.a. bias or intercept) should beadded to the decision function.",True
,"intercept_scaling intercept_scaling: float, default=1Useful only when the solver `liblinear` is usedand `self.fit_intercept` is set to `True`. In this case, `x` becomes`[x, self.intercept_scaling]`,i.e. a ""synthetic"" feature with constant value equal to`intercept_scaling` is appended to the instance vector.The intercept becomes``intercept_scaling * synthetic_feature_weight``... note:: The synthetic feature weight is subject to L1 or L2 regularization as all other features. To lessen the effect of regularization on synthetic feature weight (and therefore on the intercept) `intercept_scaling` has to be increased.",1
,"class_weight class_weight: dict or 'balanced', default=NoneWeights associated with classes in the form ``{class_label: weight}``.If not given, all classes are supposed to have weight one.The ""balanced"" mode uses the values of y to automatically adjustweights inversely proportional to class frequencies in the input dataas ``n_samples / (n_classes * np.bincount(y))``.Note that these weights will be multiplied with sample_weight (passedthrough the fit method) if sample_weight is specified... versionadded:: 0.17 *class_weight='balanced'*",None
,"random_state random_state: int, RandomState instance, default=NoneUsed when ``solver`` == 'sag', 'saga' or 'liblinear' to shuffle thedata. See :term:`Glossary ` for details.",None
,"solver solver: {'lbfgs', 'liblinear', 'newton-cg', 'newton-cholesky', 'sag', 'saga'}, default='lbfgs'Algorithm to use in the optimization problem. Default is 'lbfgs'.To choose a solver, you might want to consider the following aspects:- 'lbfgs' is a good default solver because it works reasonably well for a wide class of problems.- For :term:`mul

In [ ]:
print(metrics.classification_report(y_test,lr_model.predict(X_test_rfe_lr)))
print(metrics.accuracy_score(y_test,lr_model.predict(X_test_rfe_lr)))


              precision    recall  f1-score   support

           1       0.97      0.91      0.94       511
           2       0.97      0.94      0.96       587
           3       0.97      0.92      0.95       331
           4       0.85      0.95      0.90       379
           5       0.95      0.96      0.96       604
           6       0.90      0.94      0.92       370

    accuracy                           0.94      2782
   macro avg       0.94      0.94      0.94      2782
weighted avg       0.94      0.94      0.94      2782

0.9392523364485982
